In [ ]:
import torch.nn as nn
import torch
import torch.nn as nn
import numpy as np
import os
import cv2

import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')

from Scripts.Utils.Imports import *
from Scripts.Models.AttentionModel import *
import Scripts.cifar10_DataLoader as DL
import Modules.Utils.ModelWrapperGenerator as MW


In [ ]:
# If use GPU
device = torch.device(GetLowestGPU(pick_from=[0]))
# If use CPU
# device = torch.device("cpu")

# helper functions
def to_torch(x):
    return torch.from_numpy(x).float().to(device)
def to_numpy(x):
    return x.detach().cpu().numpy()

In [ ]:
p_train = np.random.permutation(50000)
p_val = np.random.permutation(10000)

path = 'data'
train_dataset = DL.DataLoader(p_train, path, train=True)
val_dataset = DL.DataLoader(p_val, path, train=False)

In [ ]:
lr = 0.001
criterion = nn.CrossEntropyLoss()

def Radar_BCE(pred, true):
    return nn.functional.binary_cross_entropy(pred, true)

cnn = CNN_with_Attention(input_features=3, output_features=10)
cnn.to(device)
opt = torch.optim.Adam(cnn.parameters(), lr=lr)

model = MW.ModelWrapper(
        model=cnn,
        optimizer=opt,
        loss=criterion,
        save_name='../Weights/attention_demo3',
        device=device)

# train
epochs = 100
batch_size = 64
workers = 4
model.fit(
    train_dataset=train_dataset,
    batch_size=batch_size,
    epochs=epochs,
    # verbose=1,
    validation_dataset=val_dataset,
    workers=workers)